In [1]:
cd source/optimizing/caching/

/home/jake/projects/notebook/faster-python-programs/source/optimizing/caching


In [2]:
ls

cache_deterministic.py  cache_non_deterministic.py  get_key.py


In [6]:
# %load cache_deterministic.py
# file: cache_deterministic.py
# form Ziade 2008

"""Example for a deterministic cache
"""

import functools


from get_key import get_key                                     #1

cache = {}                                                      #2


def memoize_deterministic(get_key=get_key, cache=cache):         #3
    """Parameterized decorator for memoizing.
    """

    def _memoize(function):                                     #4
        """This takes the function.
        """

        @functools.wraps(function)
        def __memoize(*args, **kw):                             #5
            """This replaces the original function.
            """
            key = get_key(function, *args, **kw)                #6
            try:
                return cache[key]                               #7
            except KeyError:
                value = function(*args, **kw)                   #8
                cache[key] = value                              #9
                return value                                    #10
        return __memoize
    return _memoize


In [5]:
# %load get_key.py
# file: get_key.py
# based on Ziade 2008

"""Generate a unique key for a function and its arguments.
"""


def get_key(function, *args, **kw):                             #1
    """Make key from module and function names as well as arguments.
    """
    key = '%s.%s:' % (function.__module__,
                      function.__name__)                        #2
    hash_args = [str(arg) for arg in args]                      #3
    hash_kw = ['%s:%s' % (k, str(v))
               for k, v in kw.items()]                          #4
    return '%s::%s::%s' % (key, hash_args, hash_kw)             #5


In [7]:
@memoize_deterministic()
def add(a,b):
    print('adding')
    return a + b

In [8]:
add(2, 3)

adding


5

In [9]:
add(2, b=3)

adding


5

In [ ]:
# %load cache_non_deterministic.py
# file: cache_non_deterministic.py
# form Ziade 2008

"""Example for a cache that expires.
"""

import functools
import time

from get_key import get_key

cache = {}

def memoize_non_deterministic(get_key=get_key, storage=cache,
                             age=0):                            #1
    """Parameterized decorator that takes an expiration age.
    """
    
    def _memoize(function):
        """This takes the function.
        """

        @functools.wraps(function)
        def __memoize(*args, **kw):
            """This replaces the original function.
            """
            key = get_key(function, *args, **kw)
            try:
                value_age, value = storage[key]                 #2
                deprecated = (age != 0 and 
                             (value_age + age) < time.time())   #3
            except KeyError:
                deprecated = True                               #4
            if not deprecated:
                return value                                    #5
            storage[key] = time.time(), function(*args, **kw)   #6
            return storage[key][1]                              #7
        return __memoize
    return _memoize


In [11]:
from functools import lru_cache

In [13]:
lru_cache?

In [15]:
@lru_cache(maxsize=3)
def add(a,b):
    print('adding')
    return a + b

In [16]:
add(2,3)

adding


5

In [17]:
add.cache_info()

CacheInfo(hits=0, misses=1, maxsize=3, currsize=1)

In [18]:
add(2,3)

5

In [19]:
add.cache_info()

CacheInfo(hits=1, misses=1, maxsize=3, currsize=1)